In [1]:
import os
import cv2
from tkinter import Tk
from tkinter.filedialog import askdirectory
from tqdm.notebook import tqdm
from time import sleep

# 開啟資料夾對話框，讓使用者選擇資料夾
Tk().withdraw()
foldername = askdirectory(title='選擇資料夾')

# 取得資料夾內的所有影片檔案
video_files = [f for f in os.listdir(foldername) if f.endswith('.MP4')]
# 輸出資料夾設定
output_dir = askdirectory(title='選擇輸出資料夾')

for filename in tqdm(video_files, desc='total_file'):
    # 影片檔案的完整路徑
    filepath = os.path.join(foldername, filename)

    # 讀取影片
    cap = cv2.VideoCapture(filepath)

    # 獲取原始影片的尺寸
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 新的裁切後的尺寸
    new_width = width
    new_height = height // 4  # 裁去上四分之一和下四分之一

    # 計算裁切後的上下邊界
    top = int(height // (8/3))
    bottom = top + new_height

    

    # 輸出影片設定
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    filename_without_ext = os.path.splitext(filename)[0]
    output_filename = os.path.join(output_dir, f"{filename_without_ext}_cropped.mp4")
    output = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, new_height))

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 使用 tqdm 顯示進度條
    with tqdm(total=total_frames, desc=filename) as pbar2:
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # 裁切畫面大小
            cropped_frame = frame[top:bottom, :]

            # # 顯示裁切後的畫面
            # cv2.imshow('Cropped Frame', cropped_frame)

            # 輸出裁切後的影片
            output.write(cropped_frame)

            # 更新進度條
            pbar2.update(1)
            sleep(0.001)

            # 按下'q'鍵退出
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # 釋放資源
    cap.release()
    output.release()

cv2.destroyAllWindows()

total_file:   0%|          | 0/45 [00:00<?, ?it/s]

0519_G291_CTZ.MP4:   0%|          | 0/12336 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
import os
import cv2
from tkinter import Tk
from tkinter.filedialog import askdirectory
from tqdm.notebook import tqdm
from time import sleep
from multiprocessing import Pool, cpu_count

# 單一影片處理函式
def process_video(filename):
    # 影片檔案的完整路徑
    filepath = os.path.join(foldername, filename)

    # 讀取影片
    cap = cv2.VideoCapture(filepath)

    # 獲取原始影片的尺寸
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 新的裁切後的尺寸
    new_width = width
    new_height = height // 4  # 裁去上四分之一和下四分之一

    # 計算裁切後的上下邊界
    top = int(height // (8/3))
    bottom = top + new_height

    # 輸出影片設定
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    filename_without_ext = os.path.splitext(filename)[0]
    output_filename = os.path.join(output_dir, f"{filename_without_ext}_cropped.mp4")
    output = cv2.VideoWriter(output_filename, fourcc, fps, (new_width, new_height))

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 使用 tqdm 顯示進度條
    with tqdm(total=total_frames, desc=filename) as pbar2:
        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # 裁切畫面大小
            cropped_frame = frame[top:bottom, :]

            # 輸出裁切後的影片
            output.write(cropped_frame)

            # 更新進度條
            pbar2.update(1)
            sleep(0.001)

            # 按下'q'鍵退出
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # 釋放資源
    cap.release()
    output.release()
    cv2.destroyAllWindows()

# 開啟資料夾對話框，讓使用者選擇資料夾
Tk().withdraw()
foldername = askdirectory(title='選擇資料夾')

# 取得資料夾內的所有影片檔案
video_files = [f for f in os.listdir(foldername) if f.endswith('.MP4')]
# 輸出資料夾設定
output_dir = askdirectory(title='選擇輸出資料夾')

# 創建處理程序池
pool = Pool(cpu_count())

for filename in tqdm(video_files, desc='total_file'):
    pool.apply_async(process_video, args=(filename,))

# 等待所有處理程序結束
pool.close()
pool.join()


total_file:   0%|          | 0/45 [00:00<?, ?it/s]